In [1]:
import pandas as pd
from tqdm import tqdm
import random

In [34]:
root_path = 'data/root'
train_path = os.path.join(root_path, 'train')
test_path = os.path.join(root_path, 'test')
mortality_path = 'data/in-hospital-mortality'

In [3]:
if not os.path.exists(mortality_path):
    os.makedirs(mortality_path)

# 加载病人3的诊断信息

In [4]:
patient_id = 3
patient_path = os.path.join(train_path, str(patient_id))
diagnose_path = os.path.join(patient_path, 'episode1.csv')
label_df = pd.read_csv(diagnose_path)
label_df

,Icustay,Ethnicity,Gender,Age,Height,Weight,Length of Stay,Mortality,Diagnosis 4019,Diagnosis 4280,...,Diagnosis 570,Diagnosis 71590,Diagnosis 2869,Diagnosis 2763,Diagnosis 5770,Diagnosis V5865,Diagnosis 99662,Diagnosis 28860,Diagnosis 36201,Diagnosis 56210
0,211552,4,2,76.575342,NaN,96.800003,6.0646,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 病人3的诊断信息里面包含有该病人是否在ICU内死亡的标志

In [5]:
label_df.iloc[0]["Mortality"]

0.0

In [13]:
""" ts_path = os.path.join(patient_path, 'episode1_timeseries.csv')
with open(ts_path) as tsfile:
    # 读取该文件内的所有内容
    ts_lines = tsfile.readlines()
    #print(ts_lines)
    header = ts_lines[0]
    print()
    print(f'header: {header}')
    ts_lines = ts_lines[1:]
    print()
    #print(ts_lines)
    for line in ts_lines:
        print(f'line: {line}')
        # 每行的第一个是进入ICU内多少个hour
        hour_inicu = float(line.split(',')[0])
        print(f'hour_inicu: {hour_inicu}') """
""" n_hours=48
ts_lines = ts_lines[1:]
event_times = [float(line.split(',')[0]) for line in ts_lines]
ts_lines = [line for (line, t) in zip(ts_lines, event_times) if -eps < t < n_hours + eps] """

" n_hours=48\nts_lines = ts_lines[1:]\nevent_times = [float(line.split(',')[0]) for line in ts_lines]\nts_lines = [line for (line, t) in zip(ts_lines, event_times) if -eps < t < n_hours + eps] "

In [15]:
""" patients = list(filter(str.isdigit, os.listdir(os.path.join(root_path, 'train'))))
print(patients) """

['11', '17', '18', '19', '20', '22', '23', '25', '28', '3', '32', '33', '4']


In [36]:
def process_partition(root_path, mortality_path, partition='train', eps=1e-6, n_hours=48):
    # 创建存储处理结果的文件夹
    # 'data/in-hospital-mortality/train'
    mortality_dir = os.path.join(mortality_path, partition)
    try:
        os.makedirs(mortality_dir)
    except:
        pass
    
    xy_pairs = []
    failed = 0
    all_records = 0
    patients = list(filter(str.isdigit, os.listdir(os.path.join(root_path, partition))))
    for patient in tqdm(patients, desc='Iterating over patients in {}'.format(partition)):
        #print()
        # 得到某个病人的数据文件夹
        patient_folder = os.path.join(root_path, partition, patient)
        # 得到病人的episode*_timeseries.csv
        patient_ts_files = list(filter(lambda x: x.find("timeseries") != -1, os.listdir(patient_folder)))

        for ts_index, ts_filename in enumerate(patient_ts_files):
            all_records += 1
            with open(os.path.join(patient_folder, ts_filename)) as tsfile:
                # 读取病人ICU和diagnose结合的信息 即episode*.csv
                lb_filename = ts_filename.replace("_timeseries", "")
                label_df = pd.read_csv(os.path.join(patient_folder, lb_filename))

                # empty label file
                if label_df.shape[0] == 0:
                    #print(f'patient_{patient} have no diagnoses')
                    failed += 1
                    continue
                # 得到该次病人入住ICU是否死亡的标志
                mortality = int(label_df.iloc[0]["Mortality"])
                los = 24.0 * label_df.iloc[0]['Length of Stay']  # in hours
                #print(f'patient_{patient} stayed in ICU {los} hours')
                if pd.isnull(los):
                    #print("\n\t(length of stay is missing)", patient, ts_filename)
                    failed += 1
                    continue
                # 排除掉病人入住ICU小于48h的数据
                if los < n_hours - eps:
                    #print(f'patient_{patient} have stayed in ICU less than 48 hours')
                    failed += 1
                    continue
                # 读取了csv的全部内容
                ts_lines = tsfile.readlines()
                # header是列名
                header = ts_lines[0]
                # ts_lines是该病人某次入住ICU的17个变量的全部内容
                ts_lines = ts_lines[1:]
                # 找到所有记录的时间
                event_times = [float(line.split(',')[0]) for line in ts_lines]
                # 筛选出记录时间在0~48h内的记录
                ts_lines = [line for (line, t) in zip(ts_lines, event_times)
                            if -eps < t < n_hours + eps]

                # no measurements in ICU
                # 没有0~48h内的记录：
                if len(ts_lines) == 0:
                    #print("\n\t(no events in ICU) ", patient, ts_filename)
                    failed += 1
                    continue

                output_ts_filename = str(patient) + "_" + str(ts_index+1) + '.csv'
                with open(os.path.join(mortality_dir, output_ts_filename), "w") as outfile:
                    outfile.write(header)
                    for line in ts_lines:
                        outfile.write(line)

                xy_pairs.append((output_ts_filename, mortality))

    print("Number of created samples:", len(xy_pairs))
    print(f'success to proess : {1-float(failed)/float(all_records)}')

    if partition == "train":
        random.shuffle(xy_pairs)
    if partition == "test":
        xy_pairs = sorted(xy_pairs)

    with open(os.path.join(mortality_path, partition+"_listfile.csv"), "w") as listfile:
        listfile.write('patient_id_i, y_true\n')
        for (x, y) in xy_pairs:
            listfile.write('{},{:d}\n'.format(x, y))


In [37]:
process_partition(root_path, mortality_path, 'train')
process_partition(root_path, mortality_path, 'test')

Iterating over patients in train: 100%|██████████| 23658/23658 [10:44<00:00, 36.74it/s]
Number of created samples: 14803
success to proess : 0.5032124281877826
